# Script to implement dynamic strawberry shoot growth

In [1]:
import openalea.plantgl.all as pgl
from openalea.mtg.algo import orders
from openalea.mtg.traversal import pre_order2, pre_order2_with_filter
from oawidgets.plantgl import PlantGL
import oawidgets.mtg as oa
from openalea.strawberry.import_mtgfile import import_mtgfile
from openalea.strawberry import visu2d
from openalea.strawberry import geometry


%gui qt
%pdb

Automatic pdb calling has been turned ON


In [2]:
g=import_mtgfile(filename= ["Capriss"])
g.properties()['order'] = orders(g)


In [3]:
def plant_ids(g):
    """ the vid of all plants in g

    Args:
        g: an mtg

    Returns: (list of int) the vid of plants in g

    """
    vids = g.component_roots_at_scale(g.root, 1)
    if len(vids) > 0:
        labels = g.property('label')
        return [vid for vid in vids if labels[vid].startswith('P')]
    else:
        return vids
    
plante_ids =plant_ids(g)

In [4]:
scene=visu2d.plot2d(g, vids=[plante_ids[10]],display=False)
PlantGL(scene,group_by_color=False)

Plot(antialias=3, axes=['x', 'y', 'z'], axes_helper=1.0, background_color=16777215, camera_animation=[], camer…

In [5]:
pgl.Viewer.display(scene)

# Add dynamic properties on the mtg to simulate developpement
- phyllochrone:  intervening period between the sequential emergence of leaves 

In [6]:
def thermal_time(g, phyllochron=50):
    """add start_tt and end_tt properties on MTG according to phyllochrone
       The aim is to determine from the phyllochron a number of leaves that have appeared on the final MTG 
       and to calculate a delta that fixes the step of appearance delta_t= phyllochron / number of leaves present 
       in order to include a dynamic of appearance 
       
       Pb: phyllochrone give the number of leaves displays
       
    Parameters
    ----------
    g : Object
        An MTG
    phyllochrone : int or float,
        the intervening period between the sequential emergence of leaves

    Returns
    -------
    Object
        An MTG containing start_tt and end_tt properties
    """
    plants= g.vertices(scale=1) # plantids
    #module_scale = 2 #module scale
    max_scale = g.max_scale() # echelle la plus elevé
    my_scale = max_scale
    
    for plant in plants: 
        
        root_id = next(g.component_roots_at_scale_iter(plant, scale=my_scale)) #vid of the first module (Trunk)
        #time=0 # init a count variable
        #last_time = time + phyllochrone # last time determine the vid of the last components
        
        for vid in pre_order2(g, root_id):
            pid = g.parent(vid)
            if pid is not None:
                time = g.node(pid).end_tt
            else:
                time = 0 # to improve
            
            #time_end = time + phyllochron
            n= g.node(vid)
            n.start_tt=time
            n.end_tt=time + phyllochron
            
            '''
            components = n.components() # components of module
            nb_components= n.nb_components() # number of component of the module
            
            last_node = time_end
            
            if nb_components!=0:
                delta_time = phyllochron / last_node # delta time = time interval between components to reach phyllochrone
                for component in components:          
                    component.start_tt = last_node - delta_time # from de last node start of node
                    component.end_tt = last_node # time for the last node
                    last_node -= delta_time
                   # print(time, "plant_id :",plant,"module_id: ",module,"node_id :",component, "label :",component.label, "start_tt :",component.start_tt)
            time +=phyllochron
            '''
    return g


thermal_time(g,phyllochron=5)
#g.property("start_tt")

* strawberry visitor 3d

In [7]:
turtle=pgl.PglTurtle()

def strawberry_visitor3d(g,v,turtle, time):
    nid= g.node(v)
    geoms = geometry.get_symbols()
    label= nid.label
    turtle=turtle
    turtle.setWidth(0.01)

    if nid.start_tt<=time<nid.end_tt:
        #print(nid.start_tt,time,nid.end_tt)
        if g.edge_type(v)== "+":
            turtle.down(30)
        elif label in ("F","f"):
            turtle.rollL(geometry.roll_angle)
            #print(v,":",turtle.rollL(geometry.roll_angle))
    else:
        if g.edge_type(v)== "+":
            turtle.down(30)
        elif label in ("F","f"):
            turtle.rollL(geometry.roll_angle)
    
    v,turtle.setId(v)
    geoms.get(label)(g, v, turtle)



strawberry_visitor3d(g,v=3,turtle=turtle,time=0)


* strawberry visitor2d


In [8]:

def strawberry_visitor2d(g, v, turtle, time):
    geoms = geometry.get_symbols2d()
    turtle.setWidth(0.01)
    nid = g.node(v)
    label = g.label(v)
    draw_it = nid.drawable
    branch_ratio = nid.branch_ratio

    if nid.start_tt<=time<nid.end_tt:
        if label in ('F','f'):
            turtle.rollL(180)

        turtle.setId(v)

        if not draw_it:
            pass
        elif (label == 'F'):
            if visu2d.is_visible(g, v):
                if visu2d.type_of_crown(v, g) == 3:
                    angle = 30.
                    length = 0.5
                else:
                    angle = 90.
                    length = 1.5 * branch_ratio

                turtle.down(angle)
                turtle.F(length)
                turtle.down(-angle)
                
        elif label == 'bt':
            turtle.down(30.)
            turtle.f(0.05)
        elif label == 'HT':
            turtle.F(0.1)

        elif label == 's':
            turtle.rollL(180)
            turtle.f(0.05)
    else:
        if label in ('F','f'):
            turtle.rollL(180)

        turtle.setId(v)

        if not draw_it:
            pass
        elif (label == 'F'):
            if visu2d.is_visible(g, v):
                if visu2d.type_of_crown(v, g) == 3:
                    angle = 30.
                    length = 0.5
                else:
                    angle = 90.
                    length = 1.5 * branch_ratio

                turtle.down(angle)
                turtle.F(length)
                turtle.down(-angle)
                
        elif label == 'bt':
            turtle.down(30.)
            turtle.f(0.05)
        elif label == 'HT':
            turtle.F(0.1)

        elif label == 's':
            turtle.rollL(180)
            turtle.f(0.05)
        turtle.setId(v)
        geoms.get(label)(g, v, turtle)
        


In [9]:
def traverse_with_turtle_time(g, vid, time, visitor):
    turtle = pgl.PglTurtle()
    def push_turtle(v):
        n = g.node(v)
        try:
            start_tt = n.start_tt
            if start_tt > time:
                return False
        except: 
            pass
        if g.edge_type(v) == '+':
            turtle.push()
        return True

    def pop_turtle(v):
        n = g.node(v)
        try:
            start_tt = n.start_tt
            if start_tt > time:
                return False
        except: 
            pass
        if g.edge_type(v) == '+':
            turtle.pop()

    if g.node(vid).start_tt <= time:
        visitor(g,vid,turtle,time)

    for v in pre_order2_with_filter(g, vid, None, push_turtle, pop_turtle):
        if v == vid: continue
        # Done for the leaves
        if g.node(v).start_tt > time:
            print('Do not consider ', v, time)
            continue
        visitor(g,v,turtle,time)
    scene=turtle.getScene()
    return scene



In [10]:
def plot(g,phyllochron=1,plant=30,time_start=0,time_end=180, step=2,visitor=strawberry_visitor2d):
    plantid= [vid for vid in g.vertices(scale=1)]
    times=[x for x in range(time_start,time_end,step)]
    
    thermal_time(g,phyllochron=phyllochron)
    for time in times:
        vids=g.component_roots_at_scale(plantid[plant],scale=3)
        visu2d.color_code(g,complete=False)
        for vid in vids:
            scene= traverse_with_turtle_time(g,vid=vid,visitor=visitor,time=time)

        pgl.Viewer.display(scene)
        #PlantGL(scene, group_by_color=True)


In [11]:
plot(g,phyllochron=10,plant=48,time_start=0,time_end=400,step=2,visitor=strawberry_visitor2d)


In [12]:
from openalea.mtg.algo import split
cap=split(g)
oa.plot(cap[0])